## Classifica a qualidade do ar

- Dados da qualidade do ar: https://archive.ics.uci.edu/dataset/360/air+quality

O conjunto de dados contém 9358 instâncias de respostas médias horárias de uma matriz de 5 sensores químicos de óxido de metal incorporados em um dispositivo multissensor químico de qualidade do ar. O dispositivo estava localizado no campo em uma área significativamente poluída, no nível da estrada, dentro de uma cidade italiana. Os dados foram registrados de março de 2004 a fevereiro de 2005 (um ano), representando as gravações mais longas disponíveis gratuitamente de respostas de dispositivos de sensores químicos de qualidade do ar implantados em campo. Concentrações médias horárias do Ground Truth para CO, hidrocarbonetos não metânicos, benzeno, óxidos de nitrogênio totais (NOx) e dióxido de nitrogênio (NO2) foram fornecidas por um analisador certificado de referência co-localizado. Evidências de sensibilidades cruzadas, bem como desvios de conceito e sensor estão presentes, conforme descrito em De Vito et al., Sens. And Act. B, Vol. 129,2,2008 (citação necessária), eventualmente afetando as capacidades de estimativa de concentração dos sensores. Valores ausentes são marcados com valor -200. Este conjunto de dados pode ser usado exclusivamente para fins de pesquisa. Fins comerciais são totalmente excluídos.

